# Initialize the environment

In [ ]:
%load_ext sparkmagic.magics

In [ ]:
import os
from IPython import get_ipython
server = "http://iccluster044.iccluster.epfl.ch:8998"
username = os.environ['RENKU_USERNAME']

get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-final", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G" }}""".format(username)
)

In [ ]:
get_ipython().run_line_magic(
    "spark", f"""add -s {username}-final -l python -u {server} -k"""
)

In [ ]:
%%spark
print('We are using Spark %s' % spark.version)

# Estimation of the parameters from the data :

#### Import :

In [ ]:
%%spark

#import the isdaten df to retrieve the real arrival time
isdt = spark.read.orc('/data/sbb/part_orc/istdaten/')

#import the nodes zurich df to have the node list of interest
nodes = spark.read.orc('/group/grande_envergure/graph/nodes_zurich.orc').select('stop_id').rdd.map(lambda row: row[0].split(':')[0]).collect()

#### First data processing :

In [ ]:
%%spark
from pyspark.sql.functions import to_timestamp, col, dayofweek

#clean the data of isdt to keep only the interesting nodes
isdt_clean = isdt.where((col('bpuic').isin(nodes)
                        & (col('zusatzfahrt_tf') == 'false')
                        & (col('faellt_aus_tf') == 'false')
                        & (col('an_prognose_status') != 'UNBEKANNT')
                        & (col('ab_prognose') != ''))
                        & (col('durchfahrt_tf') == 'false')
                        & (dayofweek(to_timestamp(col('an_prognose'), "dd.MM.yyyy HH:mm:ss")) % 7 > 1))

We can now compute the delays

In [ ]:
%%spark
from pyspark.sql.functions import expr, hour

#compute the delay at each nodes and put it to zero if it the transport arrive earlier
isdt_delay = isdt_clean.withColumn("delay", to_timestamp(col('an_prognose'), "dd.MM.yyyy HH:mm:ss").cast('long')-to_timestamp(col('ankunftszeit'), "dd.MM.yyyy HH:mm").cast('long'))
isdt_delay = isdt_delay.withColumn("delay", expr("CASE WHEN delay < 0 THEN 0 ELSE delay END"))

#store the hour of the transport arrival at the stop for further preocessing
isdt_delay = isdt_delay.withColumn("hour", hour(to_timestamp(col('ankunftszeit'),"dd.MM.yyyy HH:mm"))).cache()

#store in hdfs the delay table for validation
isdt_delay_100days = isdt_delay.select('betriebstag').distinct().sample(0.05) # sample 5% of isdt_delay for validation
isdt_delay.join(isdt_delay_100days, how='inner', on='betriebstag').select(col('bpuic').alias('stop_id'),
                                                                          col('hour'),
                                                                          col('delay')).write.save("/group/grande_envergure/graph/isdt_delay.orc",format="orc", mode='overwrite'


#### Data visualization

We used this code to look how are distributed the delays at a certain random stop at a certain random hour

In [ ]:
%%spark -o delay_df

delay_df = isdt_delay.where((col('bpuic') == '8591304') & (col('hour') == '20')).select('delay').alias('stop')

In [ ]:
import matplotlib.pyplot as plt

#define specific parameters for the plot
plt.rcParams['figure.figsize'] = (10,8)
plt.rcParams['font.size'] = 8
plt.style.use('fivethirtyeight')

# Plot histogram using Pandas
plt.hist(delay_df['delay'], bins='auto')
plt.xlabel('Delay')
plt.ylabel('Frequency')
plt.title('Distribution of Delays for stop : 8591304 at 20')

# Show the plot using display()
plt.show()

We can see that it would be reasonable to assume exponential distribution for the delays

#### Computing mean delays :

To compute the average delays at the different stops we decided to group the data by the stop_id (bpuic) and the hour of the day because we expect similar delay model for a same stop at a certain hour of the day

In [ ]:
%%spark

isdt_delay_mean = isdt_delay.groupBy(['bpuic','hour']).agg(expr("AVG(delay)").alias('avg_delay')).cache()

#### Saving the table :

We now save the table to hdfs to use it outside of spark later

In [ ]:
%%spark

#we select only the columns we'll use later to estimate the probability of catching a correspondance at a certain stop
isdt_delay_mean.select(col('bpuic').alias('stop_id'),
                       col('hour'),
                       col('avg_delay')).write.save("/group/grande_envergure/graph/isdt_delay_mean.orc",format="orc", mode='overwrite')

## Proba computing :

#### Import :

We first need to creat a SparkSession to import the dataframe we created before and transform it into a pandas df

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd

#create the SparkSession
spark = SparkSession.builder\
        .appName("Router") \
        .master("local") \
        .getOrCreate()

In [ ]:
#import the table from hdfs
isdt = spark.read.orc('/group/grande_envergure/graph/isdt_delay_mean.orc')

#transform it to a pandas df
df_mean = isdt.toPandas()
df_mean.sample(5)

#### Proba estimation :

In [ ]:
import numpy as np

def to_sec(hour):
    '''
    Transform a time given in a string format to second
    input:
        - string(hour): The string datetime in the format 'hh:mm:ss'
    output:
        - float(second): The number of second corresponding to the hour string
    '''
    split = hour.split(':')
    return float(split[0])*60*60 + float(split[1])*60 + float(split[2])

def exp_prob(m_delay,f_time):
    '''
    Compute the probability of having a delay greater or equal than f_time
    following a exponential distribution
    input:
        -float(m_delay) : mean delay of the exponential distribution
        -float(f_time) : maximum free time we have before the next connection
    output:
        -float(proba) : the probability of having a delay greater or equal to f_time
    '''
    if f_time == 0 :
        return 0
    #if the m_delay is zero the probability is always equal to 1
    if m_delay == 0:
        return 1.0
    return 1.0 - np.exp(-f_time / m_delay)

def trip_conf(path, stop_means):
    '''
    Compute the probability of catching every connection of a given path
    input:
        -list(dict)(path): a list of edges that compose a path
        -df(stop_means): the dataframe storing all the avg_delays
    output:
        -float(proba): porbability of catching every connection of the path
    '''
    c_trip = '' #stores the current trip id
    f_time = 0  #stores the free time we have to catch a connection         
    prev_stop_time = 0 #stores the last arrival time for the current trip
    m_delay = 0 #stores the mean delay of the last stop we took on a trip
    p = 1.0 #stores probability of making all the connections
    avg_delay = stop_means['avg_delay'].mean() #stores the total average delay
    
    #we'll go through all the edges in the path
    for edge in path:
        
        #if the edge is a walking edge
        if edge['trip_id'] == 'None':
            c_trip = 'None'
            #we compute the walking time and substract it to the actual free time available (margin of 10 sec)
            f_time-=to_sec(edge['end_time'])-to_sec(edge['start_time'])-10
            continue
        
        #if we're changing trip (that means it is a connection)
        elif edge['trip_id'] != c_trip:
            c_trip = edge['trip_id']
            
            #if it is not the first edge (not counting the walking edges)
            if prev_stop_time != 0:
                #we compute the free time we have between the previous arrival and the next departure and add it to the free time
                f_time+=to_sec(edge['start_time'])-prev_stop_time
                #compute the probability to make the connection and multiply it to the total probability
                #(we assume that the delay experienced at different connection are independant)
                p*= exp_prob(m_delay,f_time)
                #reset the free time
                f_time = 0
        
        #actualize information about the stop
        prev_stop_time = to_sec(edge['end_time'])
        
        #we check if we have an avg delay for this stop at this hour of the day in our dataframe, if not we set the avg delay to the
        #total avg delay for all the stop
        if stop_means[(stop_means['stop_id'] == edge['end_stop_id']) & (stop_means['hour'] == prev_stop_time//3600)].any(axis=0).any():
            m_delay = stop_means.loc[(stop_means['stop_id'] == edge['end_stop_id']) & (stop_means['hour'] == prev_stop_time//3600), 'avg_delay'].values[0]
        else:
            m_delay = avg_delay
            
    return p

# Validation

In [ ]:
isdt_delay = spark.read.orc('/group/grande_envergure/graph/isdt_delay.orc')
isdt_delay.show(10)

In [ ]:
import numpy as np
from pyspark.sql.functions import col

isdt_delay = isdt_delay.repartition('stop_id', 'hour').cache()

def val_prob(stop_delay, isdt_delay, f_time, num_all_stops_hours):
    '''
    Compute the probability of having a delay greater or equal than f_time
    following a exponential distribution
    input:
        -spark_df(stop_delay) : spark dataframe of delays for a certain stop & hour
        -float(f_time) : maximum free time we have before the next connection
    output:
        -float(proba) : the probability of having a delay smaller or equal to f_time
    '''

    num_valid=stop_delay.where(col('delay') <= f_time).count()
    num_all=stop_delay.count()

    # if there are no delays for this stop at this hour suppress condition on stop and hour
    # to only check whether the free time is smaller than the delays overall
    if num_all == 0:
        num_valid=isdt_delay.where(col('delay') <= f_time).count()
        num_all=num_all_stops_hours

    return num_valid/num_all

def trip_val(path, isdt_delay):
    '''
    Compute the probability of catching every connection of a given path
    input:
        -list(dict)(path): a list of edges that compose a path
        -df(df_delay): the dataframe storing all the avg_delays
    output:
        -float(proba): probability of catching every connection of the path
    '''
    c_trip = '' #stores the current trip id
    f_time = 0  #stores the free time we have to catch a connection
    prev_stop_time = 0 #stores the last arrival time for the current trip
    p = 1.0 #stores probability of making all the connections
    num_all_stops_hours = isdt_delay.count()

    #we'll go through all the edges in the path
    for edge in path:

        #if the edge is a walking edge
        if edge['trip_id'] == 'None':
            c_trip = 'None'
            #we compute the walking time and substract it to the actual free time available (margin of 10 sec)
            f_time-=to_sec(edge['end_time'])-to_sec(edge['start_time'])-10
            continue

        #if we're changing trip (that means it is a connection)
        elif edge['trip_id'] != c_trip:
            c_trip = edge['trip_id']

            #if it is not the first edge (not counting the walking edges)
            if prev_stop_time != 0:
                #we compute the free time we have between the previous arrival and the next departure and add it to the free time
                f_time+=to_sec(edge['start_time'])-prev_stop_time
                #compute the probability to make the connection and multiply it to the total probability
                #(we assume that the delay experienced at different connection are independant)
                p *= val_prob(stop_delay, isdt_delay, f_time, num_all_stops_hours)
                #reset the free time
                f_time = 0

        #actualize information about the stop
        prev_stop_time = to_sec(edge['end_time'])

        stop_delay = isdt_delay.where((col('stop_id') == edge['end_stop_id']) & (col('hour') == prev_stop_time//3600))

    return p